In [ ]:
!pip install segmentation_models

     |████████████████████████████████| 50 kB 5.3 MB/s 


In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
#drive.mount("/content/drive", force_remount=True)

path = "/content/drive/My Drive/Master_Project"
path_prefix = path

os.chdir(path)
os.listdir(path)

Mounted at /content/drive


['TransUNet-single',
 'data',
 'predictions',
 'predict_helper.ipynb',
 'Xception_Gray_flow.ipynb',
 'Xception_Event_flow.ipynb',
 'Xception_Dual_flow.ipynb',
 'old_loss',
 'Grayscale_output',
 'Event_output',
 'Xcep_loss_record_train_Gray.npy',
 'Xcep_loss_record_train_Event.npy',
 'Xcep_iou_record_test_Gray.npy',
 'Xception_Grayscale.ipynb',
 'Xcep_iou_record_test_Event.npy',
 'Xception_Event.ipynb',
 'Dual_output',
 'Xception_Dual_try.ipynb',
 'Xception_Final.ipynb',
 'Xcep_loss_record_train_Dual.npy',
 'Xception_Dual.ipynb']

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


# ==============================================================================
# =                                   param                                    =
# ==============================================================================
import numpy as np
import tensorflow as tf
import tqdm
import cv2
import copy
from tensorflow.keras import layers
from tensorflow.keras import Model
from segmentation_models.losses import bce_jaccard_loss


# ==============================================================================
# =                                   param                                    =
# ==============================================================================

def xception_dual(Input_Size):
    input = tf.keras.layers.Input((Input_Size))
    Dual_Input = tf.keras.layers.Lambda(lambda x: x / 255)(input)
    # x has range (-1, 1)

    Dual_Model = tf.keras.applications.xception.Xception(
        include_top=False, weights=None, input_tensor=Dual_Input,
        input_shape=(320, 320, 2), pooling=max
    )
    for layer in Dual_Model.layers:
        layer._name = layer._name + str('_D')

    output_1 = Dual_Model.get_layer('block2_sepconv2_bn_D').output
    output_2 = Dual_Model.get_layer('block3_sepconv2_bn_D').output
    output_3 = Dual_Model.get_layer('block4_sepconv2_bn_D').output
    output_4 = Dual_Model.get_layer('block13_sepconv2_bn_D').output
    output_5 = Dual_Model.get_layer('block14_sepconv2_bn_D').output

    decoder_0 = tf.keras.layers.Conv2DTranspose(filters=2048, kernel_size=3, strides=2, activation='relu',
                                                padding='same')(output_5)
    decoder_0 = layers.BatchNormalization()(decoder_0)
    decoder_0 = tf.image.resize(decoder_0, (tf.shape(output_5)[1], tf.shape(output_5)[2]))
    decoder_0 = tf.concat([decoder_0, output_5], 3)

    decoder_1 = tf.keras.layers.Conv2D(filters=2048, kernel_size=3, activation='relu', padding='same')(decoder_0)
    decoder_1 = tf.keras.layers.Conv2DTranspose(filters=1024, kernel_size=3, strides=2, activation='relu',
                                                padding='same')(decoder_1)
    decoder_1 = layers.BatchNormalization()(decoder_1)
    decoder_1 = tf.image.resize(decoder_1, (tf.shape(output_4)[1], tf.shape(output_4)[2]))
    decoder_1 = tf.concat([decoder_1, output_4], 3)
    #
    decoder_2 = tf.keras.layers.Conv2D(filters=1024, kernel_size=3, activation='relu', padding='same')(decoder_1)
    decoder_2 = tf.keras.layers.Conv2DTranspose(filters=728, kernel_size=3, strides=2, activation='relu',
                                                padding='same')(decoder_2)
    decoder_2 = layers.BatchNormalization()(decoder_2)
    decoder_2 = tf.image.resize(decoder_2, (tf.shape(output_3)[1], tf.shape(output_3)[2]))
    decoder_2 = tf.concat([decoder_2, output_3], 3)
    #
    decoder_3 = tf.keras.layers.Conv2D(filters=728, kernel_size=3, activation='relu', padding='same')(decoder_2)
    decoder_3 = tf.keras.layers.Conv2DTranspose(filters=256, kernel_size=3, strides=2, activation='relu',
                                                padding='same')(decoder_3)
    decoder_3 = layers.BatchNormalization()(decoder_3)
    decoder_3 = tf.image.resize(decoder_3, (tf.shape(output_2)[1], tf.shape(output_2)[2]))
    decoder_3 = tf.concat([decoder_3, output_2], 3)
    #
    decoder_4 = tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same')(decoder_3)
    decoder_4 = tf.keras.layers.Conv2DTranspose(filters=128, kernel_size=3, strides=2, activation='relu',
                                                padding='same')(decoder_4)
    decoder_4 = layers.BatchNormalization()(decoder_4)
    decoder_4 = tf.image.resize(decoder_4, (tf.shape(output_1)[1], tf.shape(output_1)[2]))
    decoder_4 = tf.concat([decoder_4, output_1], 3)
    decoder_4 = tf.image.resize(decoder_4, [320, 320])
    #
    output = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid', padding='same')(decoder_4)
    #
    model = Model(inputs=Dual_Model.input, outputs=output)

    # model.compile(optimizer='adam', loss=bce_jaccard_loss, metrics=[iou_score])

    return model

Segmentation Models: using `keras` framework.


In [ ]:
#=============== Training Process =====================

def my_mkdir(paths):
    if not isinstance(paths, (list, tuple)):
        paths = [paths]
    for path in paths:
        if not os.path.exists(path):
            os.makedirs(path)


def train(dataset_dir, load_size, batch_size, epochs, epoch_decay, lr, beta_1):
    # output_dir
    output_dir = os.path.join('Dual_output')
    my_mkdir(output_dir)

    models_path_dual = os.path.join('Dual_output', 'models')
    my_mkdir(models_path_dual)


    # ==============================================================================
    # =                                    metrics                                 =
    # ==============================================================================

    def calc_IoU(mask_1, mask_2):
        mask_1 = mask_1 > 0.3
        mask_2 = mask_2 > 0.3

        TP = mask_1 * mask_2
        TP = TP.sum()

        FP = ((mask_1 * (1 - mask_2)) + ((1 - mask_1) * mask_2)) * mask_2
        FP = FP.sum()

        FN = ((mask_1 * (1 - mask_2)) + ((1 - mask_1) * mask_2)) * mask_1
        FN = FN.sum()

        return TP / (TP + FP + FN)

    # ==============================================================================
    # =                                    data                                    =
    # ==============================================================================
    

            
    def train_gen(BS):

        train_image_gen = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True, vertical_flip=True,
                                                                          dtype='float32', rescale=1. / 255,
                                                                          brightness_range=None, fill_mode='constant',
                                                                          cval=125, zoom_range=[0.9, 1.1],
                                                                          rotation_range=180)
        
        train_event_gen = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True, vertical_flip=True,
                                                                          dtype='float32', rescale=1. / 255,
                                                                          brightness_range=None, fill_mode='constant',
                                                                          cval=125, zoom_range=[0.9, 1.1],
                                                                          rotation_range=180)

        train_gt_gen = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True, vertical_flip=True,
                                                                       dtype='float32', rescale=1. / 255,
                                                                       brightness_range=None, fill_mode='constant',
                                                                       cval=125, zoom_range=[0.9, 1.1],
                                                                       rotation_range=180)

        train_img_generator = train_image_gen.flow_from_directory(
            "/content/drive/My Drive/Master_Project/TransUNet-single/datasets/GaE_Dataset/Training/Grayscale/",
            target_size=(320, 320),
            color_mode='grayscale', class_mode=None,
            batch_size=BS, shuffle=True, seed=25,
            interpolation='bilinear')
        
        train_event_generator = train_event_gen.flow_from_directory(
            "/content/drive/My Drive/Master_Project/TransUNet-single/datasets/GaE_Dataset/Training/Events/",
            target_size=(320, 320),
            color_mode='grayscale', class_mode=None,
             batch_size=BS, shuffle=True, seed=25,
            interpolation='bilinear')
        

        train_gt_generator = train_gt_gen.flow_from_directory(
            "/content/drive/My Drive/Master_Project/TransUNet-single/datasets/GaE_Dataset/Training/Masks/",
            target_size=(320, 320),
            color_mode='grayscale', class_mode=None,
            batch_size=BS, shuffle=True, seed=25,
            interpolation='nearest')

        while True:
            img_gen = train_img_generator.next()
            event_gen = train_event_generator.next()
            gt_gen = train_gt_generator.next()
            yield img_gen, event_gen, gt_gen

    def val_gen():

        val_image_gen = tf.keras.preprocessing.image.ImageDataGenerator(dtype='float32', rescale=1. / 255)
        val_event_gen = tf.keras.preprocessing.image.ImageDataGenerator(dtype='float32', rescale=1. / 255)
        val_gt_gen = tf.keras.preprocessing.image.ImageDataGenerator(dtype='float32', rescale=1. / 255)

        val_img_generator = val_image_gen.flow_from_directory(
            "/content/drive/My Drive/Master_Project/TransUNet-single/datasets/GaE_Dataset/Validation/Grayscale/",
            target_size=(320, 320),
            color_mode='grayscale', class_mode=None, batch_size=1,
            shuffle=True, seed=25,
            interpolation='bilinear')
        
        val_event_generator = val_event_gen.flow_from_directory(
            "/content/drive/My Drive/Master_Project/TransUNet-single/datasets/GaE_Dataset/Validation/Events/",
            target_size=(320, 320),
            color_mode='grayscale', class_mode=None, batch_size=1, 
            shuffle=True, seed=25,
            interpolation='bilinear')

        val_gt_generator = val_gt_gen.flow_from_directory(
            "/content/drive/My Drive/Master_Project/TransUNet-single/datasets/GaE_Dataset/Validation/Masks/",
            target_size=(320, 320),
            color_mode='grayscale', class_mode=None, batch_size=1, shuffle=True, seed=25,
            interpolation='nearest')

        while True:
            img_gen = val_img_generator.next()
            event_gen = val_event_generator.next()
            gt_gen = val_gt_generator.next()
            yield img_gen, event_gen, gt_gen


    # ==============================================================================
    # =                                   models                                   =
    # ==============================================================================
    len_train_set = 702  # len(os.listdir(os.path.join(dataset_dir, 'Training', 'Grayscale', 'New_folder')))
    len_val_set = 97  # len(os.listdir(os.path.join(dataset_dir, 'Validation', 'Grayscale', 'New_folder')))

    dual_segmentation_model = xception_dual((320, 320, 2))


    model_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=beta_1)

    # ==============================================================================
    # =                                 train step                                 =
    # ==============================================================================

    @tf.function
    def train_step(img, img_e, gt):
        img_ = tf.convert_to_tensor(img)
        imge_ = tf.convert_to_tensor(img_e)
        gt_ = tf.convert_to_tensor(gt)
        with tf.GradientTape(persistent=True) as t_:
            img_cc = tf.concat([img_, imge_], axis=-1)
            dual_prediction = dual_segmentation_model(img_cc, training=True)
            dual_loss = bce_jaccard_loss(gt_, dual_prediction)

        model_grad = t_.gradient(dual_loss, dual_segmentation_model.trainable_variables)
        model_optimizer.apply_gradients(zip(model_grad, dual_segmentation_model.trainable_variables))

        return dual_loss, dual_prediction

    @tf.function
    def sample(img, img_e):
        img_ = tf.convert_to_tensor(img)
        imge_ = tf.convert_to_tensor(img_e)
        img_cc = tf.concat([img_, imge_], axis=-1)
        dual_prediction = dual_segmentation_model(img_cc)

        return dual_prediction



    # ==============================================================================
    # =                                    run                                     =
    # ==============================================================================

    t_generator = train_gen(batch_size)
    v_generator = val_gen()

    best_val_metric = np.NINF

    # epoch counter
    ep_cnt = tf.Variable(initial_value=0, trainable=False, dtype=tf.int64)

    # summary
    train_summary_writer = tf.summary.create_file_writer(os.path.join(output_dir, 'summaries', 'train'))

    # sample
    sample_dir = os.path.join(output_dir, 'samples_training')
    my_mkdir(sample_dir)

    # main loop
    with train_summary_writer.as_default():
        loss_record_train = [[],[],[]]
        for ep in tqdm.trange(epochs, desc='Epoch Loop'):
            # update epoch counter
            ep_cnt.assign_add(1)
            # train for an epoch
            t = tqdm.tqdm(range(int(len_train_set / batch_size)))

            for _ in t:
                img, img_e, gt = next(t_generator)
                loss, pred_temp = train_step(img, img_e, gt)
                iou = calc_IoU(pred_temp.numpy(), gt)
                loss_record_train[0].append(loss)
                loss_record_train[1].append(iou)
                t.set_description("Training loss: {}, ".format(str(np.array(loss))) + "IoU: {} |".format(str(np.array(iou))))

            val_metric = []
            for _ in range(len_val_set):
                img, img_e, gt = next(v_generator)
                prediction = sample(img, img_e).numpy()
                val_metric.append(calc_IoU(prediction, gt))

            val_metric = np.mean(np.asarray(val_metric))
            loss_record_train[2].append(val_metric)

            img = np.concatenate(
                (cv2.cvtColor(img[0], cv2.COLOR_GRAY2RGB), cv2.cvtColor(img_e[0], cv2.COLOR_GRAY2RGB), cv2.cvtColor(prediction[0], cv2.COLOR_GRAY2RGB), cv2.cvtColor(gt[0], cv2.COLOR_GRAY2RGB)),
                axis=1) * 255

            cv2.imwrite(os.path.join(sample_dir, 'epoch_{}-iter_{}.jpg'.format(str(ep).zfill(3),
                          str(model_optimizer.iterations.numpy()).zfill(9))), img)

            if val_metric >= best_val_metric:
                print(
                    'Validation metric improved from {} to {}'.format(str(best_val_metric), str(val_metric)))
                best_val_metric = copy.deepcopy(val_metric)
                dual_segmentation_model.save_weights(os.path.join(models_path_dual, 'model_{}.h5'.format(str(ep).zfill(3))),
                                                          overwrite=True, save_format=None)                                                                                                                  
            else:
                print('Validation metric did not improve from {}'.format(str(best_val_metric)))

        np.save("/content/drive/My Drive/Master_Project/Xcep_loss_record_train_Dual.npy", loss_record_train)


dataset_dir = "/content/drive/My Drive/Master_Project/TransUNet-single/datasets/GaE_Dataset"
load_size = [320, 320]
batch_size = 8
epochs = 150
epoch_decay = 25
lr = 0.001
beta_1 = 0.9
train(dataset_dir, load_size, batch_size, epochs, epoch_decay, lr, beta_1)


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#================== Testing Process ==================
def my_mkdir(paths):
    if not isinstance(paths, (list, tuple)):
        paths = [paths]
    for path in paths:
        if not os.path.exists(path):
            os.makedirs(path)

def test(dataset_dir, load_size):
    # output_dir
    output_dir = os.path.join('Dual_output')
    my_mkdir(output_dir)


    # ==============================================================================
    # =                                    metrics                                 =
    # ==============================================================================

    def calc_IoU(mask_1, mask_2):
        mask_1 = mask_1 > 0.3
        mask_2 = mask_2 > 0.3

        TP = mask_1 * mask_2
        TP = TP.sum()

        FP = ((mask_1 * (1 - mask_2)) + ((1 - mask_1) * mask_2)) * mask_2
        FP = FP.sum()

        FN = ((mask_1 * (1 - mask_2)) + ((1 - mask_1) * mask_2)) * mask_1
        FN = FN.sum()

        return TP / (TP + FP + FN)

    # ==============================================================================
    # =                                    data                                    =
    # ==============================================================================

    def test_gen():

        test_image_gen = tf.keras.preprocessing.image.ImageDataGenerator(dtype='float32', rescale=1. / 255)
        test_event_gen = tf.keras.preprocessing.image.ImageDataGenerator(dtype='float32', rescale=1. / 255)
        test_gt_gen = tf.keras.preprocessing.image.ImageDataGenerator(dtype='float32', rescale=1. / 255)

        test_img_generator = test_image_gen.flow_from_directory(
            "/content/drive/My Drive/Master_Project/TransUNet-single/datasets/GaE_Dataset/Testing/Grayscale/",
            target_size=(320, 320),
            color_mode='grayscale', class_mode=None, batch_size=1,
            shuffle=False,
            interpolation='bilinear')
        
        test_event_generator = test_event_gen.flow_from_directory(
            "/content/drive/My Drive/Master_Project/TransUNet-single/datasets/GaE_Dataset/Testing/Events/",
            target_size=(320, 320),
            color_mode='grayscale', class_mode=None, batch_size=1,
            shuffle=False,
            interpolation='bilinear')

        test_gt_generator = test_gt_gen.flow_from_directory(
            "/content/drive/My Drive/Master_Project/TransUNet-single/datasets/GaE_Dataset/Testing/Masks/",
            target_size=(320, 320),
            color_mode='grayscale', class_mode=None, batch_size=1, shuffle=False,
            interpolation='nearest')

        while True:
            img_gen = test_img_generator.next()
            img_e_gen = test_event_generator.next()
            gt_gen = test_gt_generator.next()
            yield img_gen, img_e_gen, gt_gen


    # ==============================================================================
    # =                                   models                                   =
    # ==============================================================================
    len_test_set = 361  # len(os.listdir(os.path.join(dataset_dir, 'Testing', 'Events', 'New_folder')))

    dual_segmentation_model = xception_dual((320, 320, 2))

    model_path = path + "/Dual_output/models"
    all_models = os.listdir(model_path)
    all_models.sort()

    dual_segmentation_model.load_weights(model_path + '/' + all_models[-1], by_name=False)



    # ==============================================================================
    # =                                 train step                                 =
    # ==============================================================================

    @tf.function
    def evaluate(img, img_e):
        img_ = tf.convert_to_tensor(img)
        imge_ = tf.convert_to_tensor(img_e)
        img_cc = tf.concat([img_, imge_], axis=-1)
        dual_prediction = dual_segmentation_model(img_cc)

        return dual_prediction



    # ==============================================================================
    # =                                    run                                     =
    # ==============================================================================

    te_generator = test_gen()


    # summary
    test_summary_writer = tf.summary.create_file_writer(os.path.join(output_dir, 'summaries', 'test'))

    # sample
    evaluate_dir = os.path.join(output_dir, 'test_prediction')
    my_mkdir(evaluate_dir)

    # main loop
    with test_summary_writer.as_default():
        iou_record_test = []
        for i in tqdm.trange(len_test_set, desc='Testing images'):
            img, img_e, gt = next(te_generator)
            prediction = evaluate(img, img_e).numpy()
            iou = calc_IoU(prediction, gt)
            iou_record_test.append(iou)
            print(" | Case {} ".format(str(np.array(i))) + "evaluated IoU: {}".format(str(np.array(iou))))

            img = np.concatenate(
                (cv2.cvtColor(img[0], cv2.COLOR_GRAY2RGB), cv2.cvtColor(img_e[0], cv2.COLOR_GRAY2RGB), cv2.cvtColor(prediction[0], cv2.COLOR_GRAY2RGB), cv2.cvtColor(gt[0], cv2.COLOR_GRAY2RGB)),
                axis=1) * 255

            cv2.imwrite(os.path.join(evaluate_dir, 'Image Test Case_{}.jpg'.format(str(i).zfill(3))), img)
            
        np.save("/content/drive/My Drive/Master_Project/Xcep_iou_record_test_Dual.npy", iou_record_test)


dataset_dir = "/content/drive/My Drive/Master_Project/TransUNet-single/datasets/GaE_Dataset"
load_size = [320, 320]
test(dataset_dir, load_size)

/usr/local/lib/python3.7/dist-packages/keras/applications/xception.py:133: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 2 input channels.
  weights=weights)
Testing images:   0%|          | 0/361 [00:00<?, ?it/s]

Found 361 images belonging to 1 classes.
Found 361 images belonging to 1 classes.
Found 361 images belonging to 1 classes.


Testing images:   0%|          | 1/361 [00:23<2:20:52, 23.48s/it]

 | Case 0 evaluated IoU: 0.9127121001390821


Testing images:   1%|          | 2/361 [00:25<1:06:25, 11.10s/it]

...

 | Case 358 evaluated IoU: 0.8279299847792998


Testing images: 100%|█████████▉| 360/361 [09:50<00:01,  1.56s/it]

 | Case 359 evaluated IoU: 0.7703233173934766


Testing images: 100%|██████████| 361/361 [09:52<00:00,  1.64s/it]

 | Case 360 evaluated IoU: 0.7762936956189894
